In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier

import tensorflow as tf
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TFAutoModelForSequenceClassification
from transformers import AdamWeightDecay
from datasets import Dataset, DatasetDict

print(tf.__version__)
print(tf.config.list_physical_devices())

# startegy for training on multiple gpus
mirrored_strategy = tf.distribute.MirroredStrategy()


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.11.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
path = "/kaggle/input/edos-1m/"

dataset_percentage = 0.4
dataset = pd.read_csv(path + "EDOS 1M.csv")
dataset = dataset.head(int(len(dataset)*dataset_percentage))

#dataset = dataset.head(10000)
#classes = dataset["eb+_emot"].unique()


In [3]:
#preparing the new dataset containing utterances pairs

df = dataset.merge(dataset, on='dialogue_id', how='inner') #self join
#creating auxialiary attributes
df['is_first'] = (df['turn_x'] == 1) & (df['turn_y'] == 1)
df['is_last'] = (df['turn_x'] == df['turn_y']) & (df['turn_y'] == df.groupby('dialogue_id')['turn_y'].transform(max))
#keep only first/last utterances and all the consecutive pairs               
df = df[df['is_first'] | df['is_last'] | (df['turn_x'] == df['turn_y'] - 1)]
#display(df,10) 

#df_preceding will be used to predict the last utterance, given the previous context, if it exists
df_preceding = df[df['is_last'] == 0]
df_preceding = df_preceding[['dialogue_id','turn_x','uttr_x','turn_y','uttr_y','eb+_emot_y','is_first']].rename(columns={'eb+_emot_y': 'label'})
#df_following will be used to predict the first utterance, given the following context, if it exists
df_following = df[df['is_first'] == 0]
df_following = df_following[['dialogue_id','turn_x','uttr_x','turn_y','uttr_y','eb+_emot_x','is_last']].rename(columns={'eb+_emot_x': 'label'})
#replace utterances with empty strings for first and last samples of each conversation
df_preceding.loc[df_preceding['is_first'] == 1, 'uttr_x'] = ""
df_following.loc[df_following['is_last'] == 1, 'uttr_y'] = ""

display(df_preceding,10)
display(df_following,10)

,dialogue_id,turn_x,uttr_x,turn_y,uttr_y,label,is_first
0,97,1,,1,You moron ! What fool washes diapers by the we...,angry,True
1,97,1,You moron ! What fool washes diapers by the we...,2,You useless fool !,furious,False
4,99,1,,1,How dare you sleep !,furious,True
5,99,1,How dare you sleep !,2,Up ! Go and clean the house .,prepared,False
8,100,1,,1,Clean the kitchen .,prepared,True
...,...,...,...,...,...,...,...
3569698,2871304,2,The corridor . He should be there . Thank you .,3,"Hi , newbie . This is a beginner . Look beginn...",acknowledging,False
3569702,2871328,1,,1,"Hey , what 's with the fancy car ?",questioning,True
3569703,2871328,1,"Hey , what 's with the fancy car ?",2,Perhaps a good trick .,suggesting,False
3569706,2871339,1,,1,"You - Uncle , a long time not seen you . How a...",questioning,True


10

,dialogue_id,turn_x,uttr_x,turn_y,uttr_y,label,is_last
1,97,1,You moron ! What fool washes diapers by the we...,2,You useless fool !,angry,False
3,97,2,You useless fool !,2,,furious,True
5,99,1,How dare you sleep !,2,Up ! Go and clean the house .,furious,False
7,99,2,Up ! Go and clean the house .,2,,prepared,True
9,100,1,Clean the kitchen .,2,"I cleaned the kitchen , ma 'am .",prepared,False
...,...,...,...,...,...,...,...
3569701,2871304,3,"Hi , newbie . This is a beginner . Look beginn...",3,,acknowledging,True
3569703,2871328,1,"Hey , what 's with the fancy car ?",2,Perhaps a good trick .,questioning,False
3569705,2871328,2,Perhaps a good trick .,2,,suggesting,True
3569707,2871339,1,"You - Uncle , a long time not seen you . How a...",2,I was looking for you . I see you prosper .,questioning,False


10

In [4]:
df_choice = 0 #0 for preceding, 1 for following
# train, validation and test split
if df_choice == 0:
    df = df_preceding
    train_X, valid_X, train_y, valid_y = train_test_split(df[['dialogue_id','uttr_x','uttr_y','is_first']], df['label'], test_size=0.15, stratify= None, shuffle=False)
else:
    df = df_following
    train_X, valid_X, train_y, valid_y = train_test_split(df[['dialogue_id','uttr_x','uttr_y','is_last']], df['label'], test_size=0.15, stratify= None, shuffle=False)

classes = df['label'].unique()
print(len(classes))

display(valid_X,10)
print("train size: ", len(train_X))
print("validation size: ", len(valid_X))

41


,dialogue_id,uttr_x,uttr_y,is_first
2977788,2024815,,Will he come or not ? Now will see .,True
2977789,2024815,Will he come or not ? Now will see .,I bet Malacarne won 't come .,False
2977794,2024815,I bet Malacarne won 't come .,Don Anto !,False
2977799,2024815,Don Anto !,What 's up ? It 's from your lawyer . There 's...,False
2977804,2024820,,"Ma 'am , where does Rosaria , the professor 's...",True
...,...,...,...,...
3569698,2871304,The corridor . He should be there . Thank you .,"Hi , newbie . This is a beginner . Look beginn...",False
3569702,2871328,,"Hey , what 's with the fancy car ?",True
3569703,2871328,"Hey , what 's with the fancy car ?",Perhaps a good trick .,False
3569706,2871339,,"You - Uncle , a long time not seen you . How a...",True


10

train size:  962004
validation size:  169766


In [5]:
# model metadata
model_name = "distilbert-base-uncased"
# map expected ids to their labels and viceversa
id2label = dict(zip(range(len(classes)), classes))
label2id = dict(zip(classes, range(len(classes))))
id2label


{0: 'angry',
 1: 'furious',
 2: 'prepared',
 3: 'acknowledging',
 4: 'trusting',
 5: 'confident',
 6: 'hopeful',
 7: 'caring',
 8: 'sentimental',
 9: 'anticipating',
 10: 'wishing',
 11: 'surprised',
 12: 'ashamed',
 13: 'questioning',
 14: 'sad',
 15: 'nostalgic',
 16: 'devastated',
 17: 'terrified',
 18: 'embarrassed',
 19: 'lonely',
 20: 'encouraging',
 21: 'suggesting',
 22: 'content',
 23: 'afraid',
 24: 'impressed',
 25: 'agreeing',
 26: 'apprehensive',
 27: 'proud',
 28: 'annoyed',
 29: 'anxious',
 30: 'grateful',
 31: 'excited',
 32: 'neutral',
 33: 'faithful',
 34: 'guilty',
 35: 'consoling',
 36: 'disgusted',
 37: 'disappointed',
 38: 'jealous',
 39: 'joyful',
 40: 'sympathizing'}

In [6]:
# building the datasets
if df_choice == 0:
    flag = "is_first"
else:
    flag= "is_last"
train_data = Dataset.from_pandas(pd.DataFrame({"text_1": train_X['uttr_x'],"text_2": train_X['uttr_y'], flag: train_X[flag], "label": np.argmax(pd.get_dummies(train_y).to_numpy(), axis=1)}), preserve_index=False)
valid_data = Dataset.from_pandas(pd.DataFrame({"text_1": valid_X['uttr_x'],"text_2": valid_X['uttr_y'], flag: valid_X[flag], "label": np.argmax(pd.get_dummies(valid_y).to_numpy(), axis=1)}), preserve_index=False)

# shuffling is performed at the previous operation -> we need to redefine valid_y
valid_y = valid_data['label']

data = DatasetDict()
data['train'] = train_data
data['validation'] = valid_data

print(data['train'][0])
data

{'text_1': '', 'text_2': 'You moron ! What fool washes diapers by the well !', 'is_first': True, 'label': 3}


DatasetDict({
    train: Dataset({
        features: ['text_1', 'text_2', 'is_first', 'label'],
        num_rows: 962004
    })
    validation: Dataset({
        features: ['text_1', 'text_2', 'is_first', 'label'],
        num_rows: 169766
    })
})

In [7]:
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# tokenize the data
def preprocess_function(examples):
    return tokenizer(examples["text_1"],examples["text_2"], truncation=True)

cols = data["train"].column_names
tokenized_data = data.map(preprocess_function, batched=True, remove_columns=["text_1","text_2",flag])
#okkk
print(tokenized_data["train"][0])
tokenized_data

  0%|          | 0/963 [00:00<?, ?ba/s]

  0%|          | 0/170 [00:00<?, ?ba/s]

{'label': 3, 'input_ids': [101, 102, 2017, 22822, 2239, 999, 2054, 7966, 9378, 2229, 22939, 7347, 2011, 1996, 2092, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 962004
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 169766
    })
})

In [8]:
# metadata
batch_size = 16
num_epochs = 10
batches_per_epoch = len(tokenized_data["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
print("Total train steps: " + str(total_train_steps))
print("Batches per epoch: " + str(batches_per_epoch))

#optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

data_collator = DataCollatorWithPadding(tokenizer, return_tensors="tf")
only_fine_tune = True

# convert datasets to a suitable format for tensorflow
tf_train_dataset = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_dataset = tokenized_data["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator
)

num_train_steps = len(tf_train_dataset) * num_epochs
print("Number of training steps: " + str(num_train_steps))

lr_scheduler = PolynomialDecay(initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps)

optimizer = Adam(learning_rate=lr_scheduler)

# create callback to save model at the end of each epoch
save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="./models/model_checkpoint",
    save_weights_only=False,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True
)
    
# apply parallel computation on kaggle
#with mirrored_strategy.scope():
    # load the model
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(classes), id2label=id2label, label2id=label2id)

# leave only the classification layer trainable
if only_fine_tune:
    for i in range(1):
        model.layers[i].trainable = False

model.compile(
    optimizer = optimizer,
    loss = SparseCategoricalCrossentropy(from_logits=True),
    metrics = ["accuracy"]
)

model.summary()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Total train steps: 75155
Batches per epoch: 15031
Number of training steps: 75155


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  31529     
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,985,001
Trainable params: 622,121
Non-trainable params: 66,362,880
_________________________________________________________________


In [ ]:
# training
history = model.fit(
          x=tf_train_dataset,
          validation_data=tf_validation_dataset,
          epochs=num_epochs,                        
          callbacks = [save_model_callback]
)

# save the model
model.save_pretrained("./models/model1")


In [ ]:
# validate the model -> accuracy should correspond to final val_accuracy
bert_y = np.argmax(model.predict(tf_validation_dataset)["logits"], axis=1)

print('Results for BERT-based classifier:')
print(classification_report(valid_y, bert_y, target_names=classes))
